Importo las librerías correspondientes: 

In [ ]:
import os
import csv
from googletrans import Translator
from httpcore import TimeoutException

Establezco la ruta de los archivos:


In [ ]:
def file_path(one_string):
    base_path = os.path.abspath(os.path.dirname("__file__"))
    archives = os.path.join(os.path.dirname(base_path), one_string)
    return archives

volcano_archives = os.path.join("datasets_input", "volcanics_input.csv")
volcano_spanish = os.path.join("datasets_output", "dataset_volcanic.csv")

file_input = file_path(volcano_archives)
file_output = file_path(volcano_spanish)


Funciones del programa:


In [ ]:
def translator_volcano_type(data):
    """Traduce el string pasado por parámetro al español"""
    translator = Translator()
    return translator.translate(data, dest='es').text.title()


def booleans_flags(data, one_string,):
    """Devuelve una lista con True en los lugares donde se encuentra
     el string pasado por parámetro, sino devuelve '¿?"""
    return True if data == one_string else '¿?'


def unifier_name_country_columns (data_name, data_country):
    """Devuelve el nombre y el país, entre paréntesis, de cada volcán"""
    return "{name} ({country})".format(name=data_name, country=data_country)


def data_volcanic_eruption(line, retries=0):
    """Escribe  una linea en .csv de salida,
     reitentando 2 veces si fallo la conexión"""
    if retries < 3:
        try: 
            new_data = (line[0], line[10], translator_volcano_type(line[9]),
                        booleans_flags(line[3], 'Tsunami'), 
                        booleans_flags(line[4], 'Earthquake'),
                        unifier_name_country_columns(line[5], line[7]))
            return new_data
        except TimeoutException:
            print("Tenemos una demora con la comunicación con Google, seguimos intentando") 
            retries += 1 
            data_volcanic_eruption(line, retries)
    else:
        raise Exception

Programa:

In [ ]:
try:
    with open(file_input, encoding="utf8") as data_set:
        reader = csv.reader(data_set, delimiter=";")
        header, data_file_input = next(reader), list(reader)

    # sobre escribo el header
    header = ['Year', 'Volcanic Explosivity Index', 'Volcano Type',
              'Flag Tsunami', 'Flag Earthquake', 'Name and country']

    # genero una lista de listas con las columnas necesarias en el archivo
    new_data = [data_volcanic_eruption(line) for line in data_file_input]
            
    # creo un archivo .csv de new_file y lo cargo
    with open(file_output, "w", encoding="utf8",  newline="") as new_file:
        writer = csv.writer(new_file)
        # agrego el nuevo encabezado
        writer.writerow(header)
        # agrego los datos
        writer.writerows(new_data)  
      
except FileNotFoundError:
    print('ERROR: No se ha encontrado el archivo.')
except Exception:
    print("Tuvimos problemas con la carga, necesita reiniciar el proceso")
else:
    print('Se ha creado el archivo correctamente.')      